In [18]:
# QUICK START
# 1. Set COLLECTION, MAX_ITEMS, and PROVIDER in Cell 4
# 2. Run all cells
# 3. Review exported CSVs:
#    - ia_pulp_snapshot.csv
#    - ia_pulp_issues_report.csv
#    - ia_pulp_ai_field_fixes_report.csv
#    - ia_pulp_proposed_cleaned.csv (optional)

In [19]:
# =============================================================================
# INSTALL DEPENDENCIES
# =============================================================================
# This installs all required Python packages:
# - pandas: Data manipulation and CSV export
# - requests: HTTP calls to Internet Archive APIs
# - tqdm: Progress bars for long-running operations
# - google-genai, openai, anthropic: LLM provider SDKs

!pip -q install pandas requests tqdm google-genai openai anthropic
print("✓ Dependencies installed")

In [20]:
# =============================================================================
# IMPORT LIBRARIES
# =============================================================================
# Standard library imports for file I/O, regex, JSON parsing, and time delays

import os
import re
import json
import time
import requests
import pandas as pd
from tqdm import tqdm
from datetime import datetime
print("✓ Libraries imported")

In [32]:
# =============================================================================
# CONFIGURATION - EDIT THESE VALUES
# =============================================================================

# -------- Demo Target (EDIT THIS) --------
COLLECTION = "magazine_rack"    # Internet Archive collection identifier
                                # Find collections at: https://archive.org/
MAX_ITEMS = 200                 # Start with 50-200 for testing, scale up later

FIELDS = [
    "identifier","title","creator","date","publisher","description",
    "subject","language","collection","mediatype"
]

# -------- Output File Names (Customize as needed) --------
SNAPSHOT_CSV = "ia_pulp_snapshot.csv"
ISSUES_CSV   = "ia_pulp_issues_report.csv"
FIXES_CSV    = "ia_pulp_ai_field_fixes_report.csv"
PROPOSED_CSV = "ia_pulp_proposed_cleaned.csv"

# -------- LLM Provider (EDIT THIS) --------
PROVIDER = "gemini"     # "gemini" | "openai" | "anthropic"

# Model names (reasonable defaults; adjust if needed)
GEMINI_MODEL   = "gemini-2.5-flash"
OPENAI_MODEL   = "gpt-4.1-mini"
ANTHROPIC_MODEL= "claude-3-5-sonnet-latest"

# -------- Rate limiting --------
SLEEP_BETWEEN_CALLS = 0.2        # Seconds between API calls
                                 # Increase to 0.5-1.0 if you hit rate limits
print(f"✓ Configuration loaded")
print(f"  Collection: {COLLECTION}")
print(f"  Max items: {MAX_ITEMS}")
print(f"  Provider: {PROVIDER}")

✓ Configuration loaded
  Collection: magazine_rack
  Max items: 200
  Provider: gemini


In [22]:
# =============================================================================
# API KEY CONFIGURATION
# =============================================================================
# ⚠️  SECURITY WARNING: Do not commit API keys to version control!
# ⚠️  Remove your key before sharing this notebook!

# Method 1: Direct assignment (for testing only - NOT SECURE)
GEMINI_API_KEY = ""  # ← Paste your API key here (get one at https://ai.google.dev/)
OPENAI_API_KEY = ""  # ← Only needed if PROVIDER = "openai"
ANTHROPIC_API_KEY = ""  # ← Only needed if PROVIDER = "anthropic"

# Method 2: Colab Secrets (RECOMMENDED - more secure)
# Uncomment these lines and add secrets via the 🔑 icon in the left sidebar:
# from google.colab import userdata
# GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
# OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
# ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')

# Validate API key is present for selected provider
assert GEMINI_API_KEY or PROVIDER != "gemini", "⚠️  Missing GEMINI_API_KEY"
assert OPENAI_API_KEY or PROVIDER != "openai", "⚠️  Missing OPENAI_API_KEY"
assert ANTHROPIC_API_KEY or PROVIDER != "anthropic", "⚠️  Missing ANTHROPIC_API_KEY"

print("✓ API keys configured")

## 📥 STEP 1: Fetch Metadata from Internet Archive

This section retrieves item metadata from the Internet Archive and saves it to a CSV file.

**What happens here:**
1. Search the collection for item identifiers
2. Fetch detailed metadata for each item
3. Normalize the metadata structure
4. Export to `ia_pulp_snapshot.csv`

**Runtime:** ~60-90 seconds for 200 items

In [23]:
# =============================================================================
# FUNCTION: Search Internet Archive for Item Identifiers
# =============================================================================
# Uses the IA Advanced Search API to get a list of item IDs from a collection

IA_ADVANCEDSEARCH = "https://archive.org/advancedsearch.php"

def ia_search_identifiers(collection: str, fields: list, max_items: int = 200) -> list:
    """
    Query Internet Archive's search API for items in a collection.

    Args:
        collection: Internet Archive collection name (e.g., "magazine_rack")
        fields: List of metadata fields to retrieve
        max_items: Maximum number of items to return

    Returns:
        List of document dictionaries containing requested fields
    """
    query = f'collection:({collection})'
    params = {
        "q": query,
        "fl[]": fields,
        "rows": max_items,
        "page": 1,
        "output": "json"
    }
    r = requests.get(IA_ADVANCEDSEARCH, params=params, timeout=60)
    r.raise_for_status()
    docs = r.json().get("response", {}).get("docs", [])
    return docs

# Execute search and show results
docs = ia_search_identifiers(COLLECTION, FIELDS, MAX_ITEMS)
print(f"✓ Found {len(docs)} items in '{COLLECTION}' collection")
if docs:
    print(f"  First identifier: {docs[0].get('identifier')}")
else:
    print("⚠️  No items found. Check your COLLECTION name.")

len(docs), docs[0].get("identifier") if docs else None

(200, 'cameroninsider2017_20171129_1621')

In [24]:
# =============================================================================
# FUNCTION: Fetch Full Metadata & Build Snapshot CSV
# =============================================================================
# Retrieves detailed metadata for each item and exports to CSV

def ia_item_metadata(identifier: str) -> dict:
    """
    Fetch complete metadata for a single Internet Archive item.

    Args:
        identifier: Unique item identifier

    Returns:
        Full metadata JSON response
    """
    url = f"https://archive.org/metadata/{identifier}"
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    return r.json()

def normalize_item_json(identifier: str, item_json: dict) -> dict:
    """
    Extract relevant fields from IA metadata response.

    Args:
        identifier: Item ID
        item_json: Raw metadata response from IA API

    Returns:
        Dictionary with normalized field names
    """
    m = item_json.get("metadata", {}) or {}

    def pick(field):
        return m.get(field)

    return {
        "identifier": identifier,
        "title": pick("title"),
        "creator": pick("creator"),
        "date": pick("date"),
        "publisher": pick("publisher"),
        "description": pick("description"),
        "subject": pick("subject"),
        "language": pick("language"),
        "collection": pick("collection"),
        "mediatype": pick("mediatype"),
    }

# Fetch metadata for all items (with progress bar)
print(f"Fetching metadata for {len(identifiers)} items...")
print(f"(This will take ~{len(identifiers) * SLEEP_BETWEEN_CALLS / 60:.1f} minutes)")

rows = []
identifiers = [d.get("identifier") for d in docs if d.get("identifier")]

for ident in tqdm(identifiers, desc="Fetching /metadata/{identifier}"):
    try:
        item = ia_item_metadata(ident)
        rows.append(normalize_item_json(ident, item))
    except Exception as e:
        print(f"⚠️  Error fetching {ident}: {e}")
        rows.append({"identifier": ident, "fetch_error": str(e)})
    time.sleep(SLEEP_BETWEEN_CALLS)

# Save to CSV
df = pd.DataFrame(rows)
df.to_csv(SNAPSHOT_CSV, index=False)
print(f"✓ Wrote {SNAPSHOT_CSV} ({len(df)} rows)")
print(f"✓ Preview:")
df.head(3)

Fetching /metadata/{identifier}: 100%|██████████| 200/200 [04:42<00:00,  1.41s/it]

✓ wrote ia_pulp_snapshot.csv  (200 rows)


,identifier,title,creator,date,publisher,description,subject,language,collection,mediatype
0,cameroninsider2017_20171129_1621,Cameron Insider (2017),University of North Carolina Wilmington,2017,"University Archives, Randall Library, Universi...",Alumni magazine for the Cameron School of Busi...,University of North Carolina Wilmington Camero...,eng,"[collegemagazines, magazine_rack]",texts
1,pub_marine-review_1917-04,pub_marine-review_1917-04,None,None,None,None,None,None,"[newsletters_inbox, newsletters, magazine_rack]",data
2,2016NEWSLETTER,2016 NEWSLETTER,"FUMC Cleveland, MS",2016-02-23,None,March Newsletter,FUMC-Cleveland,eng,"[newsletters_inbox, newsletters, magazine_rack]",data


---

## 🔍 STEP 2: Validate Metadata & Generate Issues Report

This section runs validation rules on the fetched metadata and flags quality issues.

**Validation checks:**
- Missing required fields (HIGH)
- Invalid or out-of-range dates (MEDIUM-HIGH)
- Non-standard date formats (LOW-MEDIUM)
- Whitespace issues (LOW)
- Serialized lists in fields (LOW)

**Output:** `ia_pulp_issues_report.csv`

In [25]:
# ----------------------------
# Validation configuration
# ----------------------------
REQUIRED_FIELDS = ["identifier", "title"]

DATE_PATTERNS = [
    re.compile(r"^\d{4}$"),                      # YYYY
    re.compile(r"^\d{4}-\d{2}$"),                # YYYY-MM
    re.compile(r"^\d{4}-\d{2}-\d{2}$"),          # YYYY-MM-DD (fixed)
    re.compile(r"^\d{4}s$"),                     # 1950s
    re.compile(r"^(?:circa|c\.)\s*\d{4}$", re.I) # circa 1950 / c.1950
]

INVALID_DATE_HINTS = [
    re.compile(r"\b(?:n\.d\.|no\s*date|unknown)\b", re.I),
    re.compile(r"^\s*$")
]

MIN_YEAR = 1400
MAX_YEAR = datetime.now().year + 2

WHITESPACE_RE = re.compile(r"\s{2,}")
REQUIRED_FIELDS = [...]

print("✓ Validation rules loaded")
print(f"  Required fields: {', '.join(REQUIRED_FIELDS)}")

def _is_missing(val) -> bool:
    if val is None:
        return True
    if isinstance(val, float) and pd.isna(val):
        return True
    if isinstance(val, str) and val.strip() == "":
        return True
    return False

def _stringy(val) -> str:
    if val is None:
        return ""
    return str(val)

def _has_whitespace_issues(s: str) -> bool:
    if not s:
        return False
    return (s != s.strip()) or bool(WHITESPACE_RE.search(s))

def _normalize_whitespace(s: str) -> str:
    return WHITESPACE_RE.sub(" ", s.strip())

def _looks_like_date(val: str) -> bool:
    if not val:
        return False
    v = val.strip()
    if re.search(r"\d{4}", v):
        return True
    return any(p.match(v) for p in DATE_PATTERNS)

def _date_matches_known_patterns(val: str) -> bool:
    v = val.strip()
    return any(p.match(v) for p in DATE_PATTERNS)

def _extract_year(val: str):
    m = re.search(r"(\d{4})", val)
    return int(m.group(1)) if m else None

def validate_metadata(df: pd.DataFrame) -> pd.DataFrame:
    issues = []
    df = df.copy().reset_index(drop=True)

    for i, row in df.iterrows():
        identifier = _stringy(row.get("identifier")).strip()

        # Required fields
        for field in REQUIRED_FIELDS:
            if _is_missing(row.get(field)):
                issues.append({
                    "row_index": i,
                    "identifier": identifier,
                    "issue_type": "missing_required",
                    "field": field,
                    "current_value": row.get(field),
                    "suggested_value": None,
                    "severity": "HIGH",
                    "notes": f"Required field '{field}' is missing."
                })

        # Whitespace issues
        whitespace_fields = ["title","creator","description","publisher","date","language","subject"]
        for field in whitespace_fields:
            val = row.get(field)
            if isinstance(val, str) and _has_whitespace_issues(val):
                issues.append({
                    "row_index": i,
                    "identifier": identifier,
                    "issue_type": "whitespace",
                    "field": field,
                    "current_value": val,
                    "suggested_value": _normalize_whitespace(val),
                    "severity": "LOW",
                    "notes": "Leading/trailing or repeated whitespace."
                })

        # Date validation (single sanity check)
        date_val = row.get("date")
        if isinstance(date_val, str) and _looks_like_date(date_val):
            v = date_val.strip()

            if any(p.search(v) for p in INVALID_DATE_HINTS):
                issues.append({
                    "row_index": i,
                    "identifier": identifier,
                    "issue_type": "date_invalid",
                    "field": "date",
                    "current_value": date_val,
                    "suggested_value": None,
                    "severity": "MEDIUM",
                    "notes": "Date appears missing/unknown marker."
                })
            else:
                year = _extract_year(v)
                if year is not None and (year < MIN_YEAR or year > MAX_YEAR):
                    issues.append({
                        "row_index": i,
                        "identifier": identifier,
                        "issue_type": "date_out_of_range",
                        "field": "date",
                        "current_value": date_val,
                        "suggested_value": None,
                        "severity": "MEDIUM",
                        "notes": f"Year {year} outside expected range {MIN_YEAR}-{MAX_YEAR}."
                    })

                if not _date_matches_known_patterns(v):
                    issues.append({
                        "row_index": i,
                        "identifier": identifier,
                        "issue_type": "date_format_nonstandard",
                        "field": "date",
                        "current_value": date_val,
                        "suggested_value": None,
                        "severity": "LOW",
                        "notes": "Has a year but not in preferred patterns (YYYY / YYYY-MM / YYYY-MM-DD / 1950s / circa YYYY)."
                    })

        # Serialized list-looking strings
        for field in ["subject","creator","language"]:
            val = row.get(field)
            if isinstance(val, str) and ("[" in val or "]" in val):
                issues.append({
                    "row_index": i,
                    "identifier": identifier,
                    "issue_type": "possible_list_string",
                    "field": field,
                    "current_value": val,
                    "suggested_value": None,
                    "severity": "LOW",
                    "notes": "Looks like a serialized list; consider normalizing."
                })

    issues_df = pd.DataFrame(issues)
    if not issues_df.empty:
        issues_df = issues_df.sort_values(by=["severity","issue_type","field"]).reset_index(drop=True)
    return issues_df

issues_df = validate_metadata(df)
issues_df.to_csv(ISSUES_CSV, index=False)
print(f"✓ wrote {ISSUES_CSV}  ({len(issues_df)} issues)")
issues_df.head(10)


✓ wrote ia_pulp_issues_report.csv  (61 issues)


,row_index,identifier,issue_type,field,current_value,suggested_value,severity,notes
0,30,ElPorvenirSegoviano_Segovia_VI_1767_19040217,date_format_nonstandard,date,19040217,None,LOW,Has a year but not in preferred patterns (YYYY...
1,31,ElPorvenirSegoviano_Segovia_XVI_4676_19140716,date_format_nonstandard,date,19140716,None,LOW,Has a year but not in preferred patterns (YYYY...
2,32,ElPorvenirSegoviano_Segovia_VII_2100_19050210,date_format_nonstandard,date,19050210,None,LOW,Has a year but not in preferred patterns (YYYY...
3,33,ElPorvenirSegoviano_Segovia_VI_2059_19041223,date_format_nonstandard,date,19041223,None,LOW,Has a year but not in preferred patterns (YYYY...
4,3,The_Builder_volume_10_issue_473_page_3,whitespace,description,This was a short lived periodical from the mid...,This was a short lived periodical from the mid...,LOW,Leading/trailing or repeated whitespace.
5,4,The_Builder_volume_9_issue_417_page_14,whitespace,description,The Builder was a short lived periodical that ...,The Builder was a short lived periodical that ...,LOW,Leading/trailing or repeated whitespace.
6,8,MyAtari-March-2002,whitespace,description,"MyAtari Magazine, the online magazine for Atar...","MyAtari Magazine, the online magazine for Atar...",LOW,Leading/trailing or repeated whitespace.
7,17,The_Builder_volume_5_issue_217_page_10,whitespace,description,The Builder was a short lived periodical that ...,The Builder was a short lived periodical that ...,LOW,Leading/trailing or repeated whitespace.
8,20,The_Builder_volume_2_issue_52_page_12,whitespace,description,"This is one page of The Builder, a weekly peri...","This is one page of The Builder, a weekly peri...",LOW,Leading/trailing or repeated whitespace.
9,22,The_Builder_volume_2_issue_76_page_3,whitespace,description,The Builder was a weekly magazine on architect...,The Builder was a weekly magazine on architect...,LOW,Leading/trailing or repeated whitespace.


---

## 🤖 STEP 3: Set Up LLM Providers

This section initializes the AI provider you selected in Cell 4.

**Supported providers:**
- Google Gemini (default)
- OpenAI GPT
- Anthropic Claude

In [26]:
# =============================================================================
# LLM PROVIDER ABSTRACTION
# =============================================================================
# Provides a unified interface for different LLM providers (Gemini, OpenAI, Anthropic)

class LLMProvider:
    def generate(self, prompt: str) -> str:
        raise NotImplementedError

class GeminiProvider(LLMProvider):
    def __init__(self, api_key: str, model: str, temperature: float = 0.2):
        from google import genai
        self.client = genai.Client(api_key=api_key)
        self.model = model
        self.temperature = temperature

    def generate(self, prompt: str) -> str:
        resp = self.client.models.generate_content(
            model=self.model,
            contents=prompt,
            config={"temperature": self.temperature},
        )
        return resp.text

# Initialize the selected provider
print(f"Initializing {PROVIDER.upper()} provider...")
if PROVIDER == "gemini":
    llm = GeminiProvider(GEMINI_API_KEY, GEMINI_MODEL)
elif PROVIDER == "openai":
    llm = OpenAIProvider(OPENAI_API_KEY, OPENAI_MODEL)
elif PROVIDER == "anthropic":
    llm = AnthropicProvider(ANTHROPIC_API_KEY, ANTHROPIC_MODEL)
else:
    raise ValueError(f"Unknown provider: {PROVIDER}")

print(f"✓ {PROVIDER.upper()} provider ready")

In [27]:
# =============================================================================
# AI CONTEXT CONFIGURATION
# =============================================================================
# Defines which metadata fields to include when asking the AI for suggestions


CONTEXT_FIELDS = ["identifier","title","creator","description","publisher","date","language","subject"]

def build_issue_bundles(df: pd.DataFrame, issues_df: pd.DataFrame, max_items: int = 25):
    if issues_df.empty:
        return []

    severity_order = {"HIGH": 0, "MEDIUM": 1, "LOW": 2}
    tmp = issues_df.copy()
    tmp["severity_rank"] = tmp["severity"].map(severity_order).fillna(9)

    top_ids = (
        tmp.sort_values(["severity_rank"])
           .groupby("identifier", dropna=False)
           .head(1)
           .sort_values(["severity_rank"])
           ["identifier"]
           .dropna()
           .unique()
           .tolist()
    )[:max_items]

    bundles = []
    df_indexed = df.set_index("identifier", drop=False) if "identifier" in df.columns else None

    for ident in top_ids:
        item_row = None
        if df_indexed is not None and ident in df_indexed.index:
            item_row = df_indexed.loc[ident]
            if isinstance(item_row, pd.DataFrame):
                item_row = item_row.iloc[0]
        else:
            ridx = tmp[tmp["identifier"] == ident].iloc[0]["row_index"]
            item_row = df.iloc[int(ridx)]

        context = {f: item_row.get(f, None) for f in CONTEXT_FIELDS}
        item_issues = tmp[tmp["identifier"] == ident][
            ["issue_type","field","current_value","suggested_value","severity","notes"]
        ].to_dict("records")

        bundles.append({"identifier": ident, "context": context, "issues": item_issues})

    return bundles

issue_bundles = build_issue_bundles(df, issues_df, max_items=25)
print("Items queued for AI pass:", len(issue_bundles))
issue_bundles[0]["identifier"] if issue_bundles else None

Items queued for AI pass: 25


'ElPorvenirSegoviano_Segovia_VI_1767_19040217'

In [28]:
# =============================================================================
# FUNCTION: Build AI Fix Prompt
# =============================================================================
# Constructs the prompt sent to the LLM asking for metadata corrections

def build_fix_prompt(bundle: dict) -> str:
    return f"""
You are a metadata QA assistant. Propose field-level fixes tied ONLY to the listed issues.

Rules:
- Return STRICT JSON only. No markdown. No prose outside JSON.
- Only suggest changes that directly address the issues provided.
- Do NOT invent facts not supported by context.
- If uncertain, suggest no change and explain in notes.

Output schema:
{{
  "identifier": "...",
  "fixes": [
    {{
      "field": "title|creator|description|publisher|date|language|subject|...",
      "current_value": "...",
      "suggested_value": "...",
      "issue_type": "...",
      "reason": "...",
      "confidence": 0.0,
      "requires_human_review": true
    }}
  ],
  "notes": "..."
}}

Context:
{json.dumps(bundle["context"], ensure_ascii=False)}

Issues:
{json.dumps(bundle["issues"], ensure_ascii=False)}
""".strip()

In [29]:
# =============================================================================
# UTILITY FUNCTIONS: Whitespace Normalization
# =============================================================================
# Simple deterministic fixes that don't require AI

WHITESPACE_RE = re.compile(r"\s{2,}")

def normalize_whitespace(s: str) -> str:
    return WHITESPACE_RE.sub(" ", s.strip())

def deterministic_fixes_from_issues(issues: list) -> list:
    fixes = []
    for iss in issues:
        itype = iss["issue_type"]
        field = iss["field"]
        cur = iss.get("current_value")
        sug = iss.get("suggested_value")

        if itype == "whitespace" and isinstance(cur, str):
            sug2 = sug if isinstance(sug, str) else normalize_whitespace(cur)
            if sug2 != cur:
                fixes.append({
                    "field": field,
                    "current_value": cur,
                    "suggested_value": sug2,
                    "issue_type": itype,
                    "reason": "Trimmed leading/trailing whitespace and collapsed repeated spaces.",
                    "confidence": 1.0,
                    "requires_human_review": False
                })
    return fixes

def parse_strict_json(text: str) -> dict:
    text = text.strip()
    text = re.sub(r"^```(?:json)?\s*|\s*```$", "", text, flags=re.I)
    return json.loads(text)

def ai_fixes_for_bundle(bundle: dict) -> dict:
    prompt = build_fix_prompt(bundle)
    raw = llm.generate(prompt)
    try:
        return parse_strict_json(raw)
    except Exception as e:
        return {
            "identifier": bundle["identifier"],
            "fixes": [],
            "notes": f"JSON parse failed: {e}. Raw starts: {raw[:160]}",
        }

def generate_review_assistant_fixes(bundles: list) -> pd.DataFrame:
    rows = []
    for b in tqdm(bundles, desc="AI fix pass"):
        ident = b["identifier"]
        issues = b["issues"]

        # deterministic
        for fx in deterministic_fixes_from_issues(issues):
            rows.append({"identifier": ident, **fx, "source": "deterministic"})

        # only call AI if there are non-trivial issues
        remaining = [i for i in issues if i["issue_type"] not in ("whitespace",)]
        if remaining:
            ai = ai_fixes_for_bundle(b)
            for fx in ai.get("fixes", []):
                rows.append({"identifier": ident, **fx, "source": "ai"})

        time.sleep(SLEEP_BETWEEN_CALLS)

    fixes_df = pd.DataFrame(rows)
    if not fixes_df.empty:
        fixes_df = fixes_df.sort_values(by=["identifier","source","field"]).reset_index(drop=True)
    return fixes_df

fixes_df = generate_review_assistant_fixes(issue_bundles)
fixes_df.to_csv(FIXES_CSV, index=False)
print(f"✓ wrote {FIXES_CSV}  ({len(fixes_df)} proposed fixes)")
fixes_df.head(20)

AI fix pass: 100%|██████████| 25/25 [00:17<00:00,  1.44it/s]

✓ wrote ia_pulp_ai_field_fixes_report.csv  (35 proposed fixes)


,identifier,field,current_value,suggested_value,issue_type,reason,confidence,requires_human_review,source
0,ElPorvenirSegoviano_Segovia_VII_2100_19050210,date,19050210,1905-02-10,date_format_nonstandard,The current date format 'YYYYMMDD' is non-stan...,1.0,False,ai
1,ElPorvenirSegoviano_Segovia_VI_1767_19040217,date,19040217,1904-02-17,date_format_nonstandard,The date '19040217' is not in the preferred YY...,1.0,False,ai
2,ElPorvenirSegoviano_Segovia_VI_2059_19041223,date,19041223,1904-12-23,date_format_nonstandard,The current date format 'YYYYMMDD' is non-stan...,1.0,False,ai
3,ElPorvenirSegoviano_Segovia_XVI_4676_19140716,date,19140716,1914-07-16,date_format_nonstandard,The current date format 'YYYYMMDD' is non-stan...,1.0,False,ai
4,Eugene_PCjr_Club_News_198801_Vol4No1,description,Eugene PCjr Club News 198801 Vol4No1,Eugene PCjr Club News 198801 Vol4No1,whitespace,Trimmed leading/trailing whitespace and collap...,1.0,False,deterministic
5,Eugene_PCjr_Club_News_198801_Vol4No1,title,Eugene PCjr Club News 198801 Vol4No1,Eugene PCjr Club News 198801 Vol4No1,whitespace,Trimmed leading/trailing whitespace and collap...,1.0,False,deterministic
6,Kerrang_November_7_2015_UK,description,"Kerrang! November 7, 2015 UK","Kerrang! November 7, 2015 UK",whitespace,Trimmed leading/trailing whitespace and collap...,1.0,False,deterministic
7,Kerrang_November_7_2015_UK,title,"Kerrang! November 7, 2015 UK","Kerrang! November 7, 2015 UK",whitespace,Trimmed leading/trailing whitespace and collap...,1.0,False,deterministic
8,Kickoff_October_2015,description,Kickoff October 2015,Kickoff October 2015,whitespace,Trimmed leading/trailing whitespace and collap...,1.0,False,deterministic
9,Kickoff_October_2015,title,Kickoff October 2015,Kickoff October 2015,whitespace,Trimmed leading/trailing whitespace and collap...,1.0,False,deterministic


---

## ⚙️ STEP 4: Apply Fixes & Generate Proposed Cleaned Data

This optional section applies high-confidence AI suggestions to create a "proposed" cleaned dataset.

**Important:** This does NOT overwrite the original metadata. It creates a new CSV showing what the data would look like if fixes were applied.

In [30]:
# =============================================================================
# FUNCTION: Apply Suggested Fixes to DataFrame
# =============================================================================
# Creates a "proposed" version of the data with high-confidence fixes applied

def apply_fixes_to_df(df: pd.DataFrame, fixes_df: pd.DataFrame, min_confidence: float = 0.85) -> pd.DataFrame:
    out = df.copy()

    for _, fx in fixes_df.iterrows():
        ident = str(fx["identifier"])
        field = fx["field"]
        suggested = fx["suggested_value"]

        conf = float(fx.get("confidence", 0.0))
        needs_review = bool(fx.get("requires_human_review", True))

        # policy: apply deterministic always; apply AI only if confident and not screaming for review
        if fx.get("source") == "ai":
            if needs_review:
                continue
            if conf < min_confidence:
                continue

        if "identifier" not in out.columns or field not in out.columns:
            continue

        mask = out["identifier"].astype(str) == ident
        out.loc[mask, field] = suggested

    return out

df_proposed = apply_fixes_to_df(df, fixes_df, min_confidence=0.85)
df_proposed.to_csv(PROPOSED_CSV, index=False)
print(f"✓ wrote {PROPOSED_CSV}")

✓ wrote ia_pulp_proposed_cleaned.csv


In [31]:
# =============================================================================
# DEMO FUNCTIONS: Before/After Examples
# =============================================================================
# These functions show specific examples of how fixes would look

def demo_one(identifier: str):
    before = df[df["identifier"].astype(str) == str(identifier)]
    iss = issues_df[issues_df["identifier"].astype(str) == str(identifier)]
    fx  = fixes_df[fixes_df["identifier"].astype(str) == str(identifier)]

    print("=== BEFORE ===")
    display(before[["identifier","title","creator","date","publisher","language","subject"]].head(1))

    print("\n=== ISSUES ===")
    display(iss[["severity","issue_type","field","current_value","suggested_value","notes"]])

    print("\n=== PROPOSED FIXES ===")
    display(fx[["source","issue_type","field","current_value","suggested_value","confidence","requires_human_review","reason"]])

# pick an identifier that has issues
if not issues_df.empty:
    demo_one(issues_df.iloc[0]["identifier"])

    print("✓ Demo functions ready")
print("  Call demo_one('identifier_here') to see before/after for a specific item")

=== BEFORE ===


,identifier,title,creator,date,publisher,language,subject
30,ElPorvenirSegoviano_Segovia_VI_1767_19040217,El Porvenir Segoviano - Año VI Número 1767,Imp. de F. Santiuste (Segovia),19040217,None,spa,"[españa, prensa, diario]"



=== ISSUES ===


,severity,issue_type,field,current_value,suggested_value,notes
0,LOW,date_format_nonstandard,date,19040217,None,Has a year but not in preferred patterns (YYYY...



=== PROPOSED FIXES ===


,source,issue_type,field,current_value,suggested_value,confidence,requires_human_review,reason
1,ai,date_format_nonstandard,date,19040217,1904-02-17,1.0,False,The date '19040217' is not in the preferred YY...


---

## ✅ Workflow Complete

You now have four CSV files ready for review:

1. **`ia_pulp_snapshot.csv`** - Original metadata from Internet Archive
2. **`ia_pulp_issues_report.csv`** - All detected quality issues
3. **`ia_pulp_ai_field_fixes_report.csv`** - AI-suggested fixes with confidence scores
4. **`ia_pulp_proposed_cleaned.csv`** - Preview of cleaned data (optional)

### Next Steps:

1. Download the CSV files from the Colab file browser (📁 icon on left)
2. Review issues and AI suggestions in a spreadsheet
3. Manually approve/reject individual fixes
4. For production use, implement a write-back workflow to update source metadata

### Customization:

- Change `COLLECTION` in Cell 4 to analyze different collections
- Adjust `MAX_ITEMS` to process more/fewer records
- Switch `PROVIDER` to compare different LLMs
- Modify validation rules in Cell 8 for your use case

**Remember:** This tool is designed as a review assistant, not an automated system. Always verify AI suggestions before applying them to production data.